In [1]:
import os

import pandas as pd
from sklearn.metrics import roc_auc_score

from src.train import train
from src.datasets import prepare_dataset

In [15]:
dataset_names = ['HIA_Hou', 'BBB_Martins', 'CYP3A4_Substrate_CarbonMangels']

In [16]:
model_names = ['tree', 'DGL_GCN', 'Transformer']

In [17]:
for dataset_name in dataset_names:
    for model_name in model_names:
        train_df, val_df, test_df = prepare_dataset(dataset_name, model_name)
        
        dataset_folder = os.path.join('datasets', dataset_name, model_name)
        if not os.path.exists(dataset_folder):
            os.makedirs(dataset_folder)
        train_df.to_pickle(os.path.join(dataset_folder, 'train.pickle'))
        val_df.to_pickle(os.path.join(dataset_folder, 'val.pickle'))
        test_df.to_pickle(os.path.join(dataset_folder, 'test.pickle'))

Found local copy...
Loading...
Done!


encoding drug...
unique drugs: 578


Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!


Drug Property Prediction Mode...
in total: 578 drugs
encoding drug...
unique drugs: 578
Done.
Drug Property Prediction Mode...
in total: 578 drugs
encoding drug...
unique drugs: 578
Done.
encoding drug...
unique drugs: 1975


Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!


Drug Property Prediction Mode...
in total: 2030 drugs
encoding drug...
unique drugs: 1975
Done.
Drug Property Prediction Mode...
in total: 2030 drugs
encoding drug...
unique drugs: 1975
Done.
encoding drug...
unique drugs: 667


Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!


Drug Property Prediction Mode...
in total: 670 drugs
encoding drug...
unique drugs: 667
Done.
Drug Property Prediction Mode...
in total: 670 drugs
encoding drug...
unique drugs: 667
Done.


In [18]:
model_kwargs = {}
model_kwargs['DGL_GCN'] = dict(LR=1e-3, train_epoch=100, batch_size=64)
model_kwargs['Transformer'] = model_kwargs['DGL_GCN']
model_kwargs['tree'] = {'max_depth': 5, 'random_state': 1234,}

In [19]:
auc_scores = {}

In [20]:
from DeepPurpose import CompoundPred

In [21]:
for dataset_name in dataset_names:
    auc_scores[dataset_name] = {}
    for model_name in model_names:
        model_dir = os.path.join('models', dataset_name, model_name)
        if not os.path.exists(model_dir):
            os.makedirs(model_dir)
            
        dataset_dir = os.path.join('datasets', dataset_name, model_name)
        
        if os.path.exists(os.path.join(model_dir, 'model.pt')):
            model = CompoundPred.model_pretrained(model_dir)
        else:
            model = train(model_name, 
                          train_pickle=os.path.join(dataset_dir, 'train.pickle'),
                          val_pickle=os.path.join(dataset_dir, 'val.pickle'),
                          model_dir=model_dir,
                          **model_kwargs[model_name])

        test_dataset = pd.read_pickle(os.path.join(dataset_dir, 'test.pickle'))
        y_pred =  model.predict(test_dataset)
        
        auc_scores[dataset_name][model_name] = roc_auc_score(test_dataset['Label'], y_pred)

predicting...
predicting...
predicting...
predicting...
predicting...
predicting...


In [22]:
auc_scores_df = pd.DataFrame(auc_scores)

In [23]:
auc_scores_df

,HIA_Hou,BBB_Martins,CYP3A4_Substrate_CarbonMangels
tree,0.604950,0.838461,0.575869
DGL_GCN,0.937624,0.885884,0.650624
Transformer,0.798020,0.801809,0.532754


In [24]:
auc_scores_df.mean(axis=1)

tree           0.673094
DGL_GCN        0.824711
Transformer    0.710861
dtype: float64